In [1]:
import polars as pl
import os

from tqdm import tqdm
import eikon as ek

ek.set_app_key("fd36c20b01ed4ad6b3248805ae5031749349e3c9")

In [3]:
from source.refinitiv import EikonHelper

eikon_helper = EikonHelper(cell_limit=1_000)
eikon_helper.extract()

Cell limit: 1000
Instrument limit: 0
Tickers retrieved: 2/62472
Batches remaining: 0


0it [00:00, ?it/s]


In [4]:
import polars as pl
import pandas as pd

In [6]:
df = pd.read_parquet("assets\Eikon\data.parquet", dtype_backend="numpy_nullable")
df

,Instrument,TR.ANALYTICENVEXPENDITURES.DATE,TR.DPSACTVALUE.DATE,TR.F.STDEBTCURRPORTOFLTDEBT.DATE,TR.F.EPSBASICEXCLEXORDITEMSCOMTOT.DATE,TR.CO2ESTIMATIONMETHOD,TR.F.SALEOFTANGINTANGFIXEDASSETSGL,TR.F.LIQUIDREDEMPVALUEOFREDEEMPREFSTOCKEQ.DATE,TR.NONHAZARDOUSWASTE.DATE,TR.NOXSOXEMISSIONSREDUCTION,...,TR.F.PREFSTOCKREDEEMTOT,TR.RENEWENERGYPURCHASED,TR.F.CAPEXTOT,TR.WATERWITHDRAWALTOTAL.DATE,TR.F.CASHSTINVSTTOT.DATE,TR.ANALYTICENVCONTROV.DATE,TR.ELECTRICITYPURCHASED.DATE,TR.F.EBITDA.DATE,TR.WASTERECYCLINGRATIO,TR.F.OPPROFBEFNONRECURINCEXPN
0,AAPL.OQ,2021-09-25,2023-11-02 16:30:00,2023-09-30,2023-09-30,Reported,<NA>,NaT,2021-09-25,1,...,<NA>,751032,10959.0,2021-09-25,2023-09-30,2021-09-25,2021-09-30,2023-09-30,68.0,114301.0
1,AAPL.OQ,2020-09-26,2022-10-27 16:30:00,2022-09-24,2022-09-24,Reported,<NA>,NaT,2020-09-26,1,...,<NA>,758952,10708.0,2020-09-26,2022-09-24,2020-09-26,2020-09-30,2022-09-24,70.0,119437.0
2,AAPL.OQ,2019-09-28,2021-10-28 16:30:00,2021-09-25,2021-09-25,Reported,<NA>,NaT,2019-09-28,0,...,<NA>,781704,11085.0,2019-09-28,2021-09-25,2019-09-28,2019-09-30,2021-09-25,66.0,108949.0
3,AAPL.OQ,2018-09-29,2020-10-29 16:32:00,2020-09-26,2020-09-26,Reported,<NA>,NaT,2018-09-29,0,...,<NA>,815976,7309.0,2018-09-29,2020-09-26,2018-09-29,2018-09-30,2020-09-26,67.0,66288.0
4,AAPL.OQ,2017-09-30,2019-10-30 16:30:00,2019-09-28,2019-09-28,Reported,<NA>,NaT,2017-09-30,0,...,<NA>,695808,10495.0,2017-09-30,2019-09-28,2017-09-30,2017-09-30,2019-09-28,71.0,63930.0
5,AAPL.OQ,2016-09-24,2018-11-01 16:30:00,2018-09-29,2018-09-29,Reported,<NA>,NaT,2016-09-24,0,...,<NA>,<NA>,13313.0,2016-09-24,2018-09-29,2016-09-24,2016-09-30,2018-09-29,64.0,70898.0
6,AAPL.OQ,2015-09-26,2017-11-02 16:30:00,2017-09-30,2017-09-30,Reported,<NA>,NaT,2015-09-26,0,...,<NA>,<NA>,12451.0,2015-09-26,2017-09-30,2015-09-26,2015-09-30,2017-09-30,62.0,61344.0
7,AAPL.OQ,2014-09-27,2016-10-25 16:30:00,2016-09-24,2016-09-24,Reported,<NA>,NaT,2014-09-27,0,...,<NA>,<NA>,12734.0,2014-09-27,2016-09-24,2014-09-27,2014-09-30,2016-09-24,66.575,60024.0
8,AAPL.OQ,2013-09-28,2015-10-27 16:30:00,2015-09-26,2015-09-26,Reported,<NA>,NaT,2013-09-28,0,...,<NA>,<NA>,11488.0,2013-09-28,2015-09-26,2013-09-28,2013-09-30,2015-09-26,72.58,71230.0
9,AAPL.OQ,2012-09-29,2014-10-20 16:30:00,2014-09-27,2014-09-27,Reported,<NA>,NaT,2012-09-29,0,...,<NA>,<NA>,9813.0,2012-09-29,2014-09-27,2012-09-29,2012-09-30,2014-09-27,69.742,52503.0


In [ ]:
df.dtype

In [ ]:
df.dtypes.to_frame("dtypes").to_dict()

In [ ]:
import json

with open("dtypes.json", "w") as f:
    json.dump(df.dtypes.to_dict(), f, indent=4)

In [ ]:
df[df["TR.CO2DIRECTSCOPE1"].notna()]["TR.CO2DIRECTSCOPE1.DATE"].isna().sum()

In [ ]:
df1 = df[df["TR.EXCHANGECOUNTRY"].isin(["Indonesia", "Malaysia", "Singapore", "Thailand", "Vietnam", "Philippines"])]
df1 = df1.dropna(subset=["TR.CO2DIRECTSCOPE1"])[["Instrument", "TR.CO2DIRECTSCOPE1", "TR.CO2DIRECTSCOPE1.DATE"]]
df1

In [ ]:
df1[df1["TR.CO2DIRECTSCOPE1.DATE"].isna()]

In [ ]:
df1.to_csv("refinitiv_data_1.csv", index=False)

In [ ]:
df1["Instrument"].nunique()

In [ ]:
df2 = pd.read_excel("assets\metadata\misc\Refinitiv_Environmental_SEA 150125.xlsx", header=[0,1])
df2

In [ ]:
[c for c in df2.columns.levels[0].tolist() if "CO2" in c]

In [ ]:
df2[["Identifier (RIC)", "CO2 Equivalent Emissions Direct Scope 1 YoY\nIn the last 20 FY"]]